# Playground

In [1]:
import torch
from uda import VAE, VAEConfig

In [2]:
config = VAEConfig(
    encoder_blocks=(
        (8, 16, 16, 16),
        (16, 32, 32, 32),
        (32, 64, 64, 64),
        (64, 128, 128, 128),
        (128, 256, 256, 256),
    ),
    decoder_blocks=(
        (256, 128, 128, 128),
        (128, 64, 64, 64),
        (64, 32, 32, 32),
        (32, 16, 16, 16),
        (16, 8, 8, 8),
    ),
    latent_dim=1024,
    dim=3,
)
model = VAE(
    config=config,
    n_channels=1,
    input_size=(128, 128, 128),
)

n_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"# parameters: {n_params:,}")

# parameters: 59,480,409


In [3]:
model = model.cuda()

x = torch.empty(2, 1, 128, 128, 128).normal_()
x_ = model(x.cuda()).detach().cpu()

x_.shape

torch.Size([2, 1, 128, 128, 128])

In [1]:
import torch
from uda import UNet, UNetConfig

In [3]:
config = UNetConfig(
    in_channels=1,
    out_channels=1,
    encoder_blocks=(
        (8, 16, 16, 16),
        (16, 32, 32, 32),
        (32, 64, 64, 64),
        (64, 128, 128, 128),
    ),
    decoder_blocks=(
        (128, 64, 64, 64),
        (64, 32, 32, 32),
        (32, 16, 16, 16),
        (16, 8, 8, 8),
    ),
    dim=2,
)
model = UNet(config)

n_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"# parameters: {n_params:,}")

# parameters: 798,009


In [4]:
model = model.cuda()

x = torch.empty(2, 1, 128, 128).normal_()
x_ = model(x.cuda()).detach().cpu()

x_.shape

torch.Size([2, 1, 128, 128])